In [104]:
import pandas as pd
import numpy as np
import datetime
import jieba
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] 
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False
from chinese_calendar import is_workday
from pyecharts.charts import Bar

In [3]:
login=pd.read_csv('login.csv',encoding='gbk')
login
# 没有缺失值

,user_id,login_time,login_place
0,用户3,2018/9/6 9:32,中国广东广州
1,用户3,2018/9/7 9:28,中国广东广州
2,用户3,2018/9/7 9:57,中国广东广州
3,用户3,2018/9/7 10:55,中国广东广州
4,用户3,2018/9/7 12:28,中国广东广州
...,...,...,...
387139,用户44247,2020/6/18 7:41,中国湖北武汉
387140,用户44247,2020/6/18 8:13,中国湖北
387141,用户44248,2020/6/18 9:09,中国天津
387142,用户44249,2020/6/18 9:43,中国北京


In [4]:
login.isna().sum()

user_id        0
login_time     0
login_place    0
dtype: int64

In [5]:
#一天内重复登录的行为我们将视为一次登录即可，以减少数据量

# 方法一
# data1=login['login_time'].str.split(' ',expand=True)[0]
# login=pd.concat([login,data1],axis=1)
# login.rename(columns={0:'日期'},inplace=True)
# del login['login_time']
# login

# 方法二
def str_datetime(df,column_name='login_time'):
    df[column_name]=pd.to_datetime(df[column_name])
    df[column_name]=df[column_name].apply(lambda x:x.strftime('%Y-%m-%d'))
    return df

login=str_datetime(login)
# 将同一日期用户重复登陆行为删除
login=login.drop_duplicates()
login

,user_id,login_time,login_place
0,用户3,2018-09-06,中国广东广州
1,用户3,2018-09-07,中国广东广州
5,用户3,2018-09-10,中国广东广州
8,用户3,2018-09-10,中国北京
10,用户3,2018-09-10,中国广东
...,...,...,...
387139,用户44247,2020-06-18,中国湖北武汉
387140,用户44247,2020-06-18,中国湖北
387141,用户44248,2020-06-18,中国天津
387142,用户44249,2020-06-18,中国北京


In [94]:
# 以最近的时间为基准，计算出用户每一次登录距离现在的时间
login['时间差1']=pd.to_datetime(login['login_time']).max()-pd.to_datetime(login['login_time'])
login

,user_id,login_time,login_place,时间差1,国家,省份,地区
0,用户3,2018-09-06,中国广东广州,651 days,中国,广东,广州
1,用户3,2018-09-07,中国广东广州,650 days,中国,广东,广州
2,用户3,2018-09-10,中国广东广州,647 days,中国,广东,广州
3,用户3,2018-09-10,中国北京,647 days,中国,北京,
4,用户3,2018-09-10,中国广东,647 days,中国,广东,
...,...,...,...,...,...,...,...
267720,用户44247,2020-06-18,中国湖北武汉,0 days,中国,湖北,武汉
267721,用户44247,2020-06-18,中国湖北,0 days,中国,湖北,
267722,用户44248,2020-06-18,中国天津,0 days,中国,天津,
267723,用户44249,2020-06-18,中国北京,0 days,中国,北京,


In [95]:
# 重新排序
login=login.reset_index()
del login['index']
# # 地区拆分
# # 运行时间很长-而且切词并不准确，很多没有切除干净
# for i in range(login.shape[0]):
#     li=[str(i.word) for i in HanLP.newSegment("crf").seg(login.iloc[i,2])]
# #     li=[word for word in jieba.cut(login.iloc[i,2])]
#     if len(li)==1:
#         if '中国' in login.iloc[i,2]:
#             login.loc[i,'国家']=login.iloc[i,2][0:2]
#             login.loc[i,'省份']=login.iloc[i,2][2:]
#         else:
#             login.loc[i,'国家']=login.iloc[i,2]
#     elif len(li)==2:
#         login.loc[i,'国家']=li[0]
#         login.loc[i,'省份']=li[1]
#     else:
#         login.loc[i,'国家']=li[0]
#         login.loc[i,'省份']=li[1]
#         login.loc[i,'地区']=li[2]
#     if i % 10000 ==0:
#         print(i)
 
# 改进版本
for i in range(login.shape[0]):
    if login.loc[i,'login_place'][0:2]=='中国':
        login.loc[i,'国家']='中国'
        if '黑龙江' in login.loc[i,'login_place']:
            login.loc[i,'省份']='黑龙江'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        if '新疆维吾尔' in login.loc[i,'login_place']:
            login.loc[i,'省份']='新疆维吾尔'
            if len(login.loc[i,'login_place'])>7:
                login.loc[i,'地区']=login.loc[i,'login_place'][7:]
            else:pass
        if '内蒙古' in login.loc[i,'login_place']:
            login.loc[i,'省份']='内蒙古'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        else:
            login.loc[i,'省份']=login.loc[i,'login_place'][2:4]
            login.loc[i,'地区']=login.loc[i,'login_place'][4:]   
    else:
        li=[word for word in jieba.cut(login.iloc[i,2])]
        if len(li)==2:
            login.loc[i,'国家']=li[0]
            login.loc[i,'省份']=li[1]
        else:
            login.loc[i,'国家']=li[0]
    if i%10000==0:
        print(f'{round(i*100/(int(login.shape[0])),2)}%')

0.0%
3.74%
7.47%
11.21%
14.94%
18.68%
22.41%
26.15%
29.88%
33.62%
37.35%
41.09%
44.82%
48.56%
52.29%
56.03%
59.76%
63.5%
67.23%
70.97%
74.7%
78.44%
82.17%
85.91%
89.64%
93.38%
97.11%


In [96]:
login.to_excel('用户地区切割.xlsx')

In [97]:
# 重新读取已经切分好城市的数据
login =pd.read_excel('用户地区切割.xlsx',index_col=0)
login

,user_id,login_time,login_place,时间差1,国家,省份,地区
0,用户3,2018-09-06,中国广东广州,651,中国,广东,广州
1,用户3,2018-09-07,中国广东广州,650,中国,广东,广州
2,用户3,2018-09-10,中国广东广州,647,中国,广东,广州
3,用户3,2018-09-10,中国北京,647,中国,北京,NaN
4,用户3,2018-09-10,中国广东,647,中国,广东,NaN
...,...,...,...,...,...,...,...
267720,用户44247,2020-06-18,中国湖北武汉,0,中国,湖北,武汉
267721,用户44247,2020-06-18,中国湖北,0,中国,湖北,NaN
267722,用户44248,2020-06-18,中国天津,0,中国,天津,NaN
267723,用户44249,2020-06-18,中国北京,0,中国,北京,NaN


In [98]:
stu_info=pd.read_csv('study_information.csv',encoding='gbk')
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020/4/21 10:11,width: 0%;,0.0
1,用户3,课程136,2020/3/5 11:44,width: 1%;,0.0
2,用户3,课程205,2018/9/10 18:17,width: 63%;,0.0
3,用户4,课程26,2020/3/31 10:52,width: 0%;,319.0
4,用户4,课程34,2020/3/31 10:52,width: 0%;,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020/6/17 22:16,width: 0%;,0.0
194970,用户44246,课程76,2020/6/17 22:36,width: 4%;,0.0
194971,用户44247,课程19,2020/6/18 7:54,width: 100%;,499.0
194972,用户44247,课程32,2020/6/18 7:41,width: 0%;,499.0


In [10]:
stu_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 7.4+ MB


In [11]:
stu_info=stu_info.drop_duplicates()
stu_info.info()
#没有完全重复数据,但是存在价格缺失数据

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


In [12]:
stu_info[stu_info.price.isnull()].course_id.value_counts()

课程51    4011
课程96     227
Name: course_id, dtype: int64

In [13]:
stu_course=stu_info.groupby(['course_id']).agg({'price':['max','min']})
stu_course[(stu_course['price']['max']-stu_course['price']['min'])!=0]
#不存在差异定价,且51和96课程价格为nan，其中51和96课程共计4238条数据，暂时不做处理

price    
            max min
course_id          
课程51        NaN NaN
课程96        NaN NaN

In [14]:
# 将进度转化成为数值型，从而便于计算
stu_info['learn_process']=stu_info['learn_process'].apply(lambda x:int(x.split('width:')[-1].split('%')[0]))
# 对加入时间进行相同操作
stu_info=str_datetime(stu_info,column_name='course_join_time')
stu_info

,user_id,course_id,course_join_time,learn_process,price
0,用户3,课程106,2020-04-21,0,0.0
1,用户3,课程136,2020-03-05,1,0.0
2,用户3,课程205,2018-09-10,63,0.0
3,用户4,课程26,2020-03-31,0,319.0
4,用户4,课程34,2020-03-31,0,299.0
...,...,...,...,...,...
194969,用户44245,课程76,2020-06-17,0,0.0
194970,用户44246,课程76,2020-06-17,4,0.0
194971,用户44247,课程19,2020-06-18,100,499.0
194972,用户44247,课程32,2020-06-18,0,499.0


In [27]:
users=pd.read_csv('users.csv',encoding='gbk')
users

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school
0,0,用户44251,2020/6/18 9:49,2020/6/18 9:49,0,0,41.25,NaN
1,1,用户44250,2020/6/18 9:47,2020/6/18 9:48,0,0,0,NaN
2,2,用户44249,2020/6/18 9:43,2020/6/18 9:43,0,0,16.22,NaN
3,3,用户44248,2020/6/18 9:09,2020/6/18 9:09,0,0,0,NaN
4,4,用户44247,2020/6/18 7:41,2020/6/18 8:15,0,0,1.8,NaN
...,...,...,...,...,...,...,...,...
43911,43978,用户6,2018/9/11 16:13,2018/9/11 16:14,0,0,0,NaN
43912,43979,用户5,2018/9/10 15:48,2020/6/15 17:13,9,5,"2,116.15",NaN
43913,43980,用户4,2018/9/10 14:15,2020/6/5 9:50,4,0,341.2,NaN
43914,43981,用户3,2018/9/4 13:32,2020/6/18 9:18,2,1,370.35,NaN


In [28]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43916 entries, 0 to 43915
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              43916 non-null  int64 
 1   user_id                 43916 non-null  object
 2   register_time           43916 non-null  object
 3   recently_logged         43724 non-null  object
 4   number_of_classes_join  43916 non-null  int64 
 5   number_of_classes_out   43916 non-null  int64 
 6   learn_time              43916 non-null  object
 7   school                  10569 non-null  object
dtypes: int64(3), object(5)
memory usage: 2.7+ MB


In [29]:
users=users[~users.user_id.isnull()]
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43916 entries, 0 to 43915
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              43916 non-null  int64 
 1   user_id                 43916 non-null  object
 2   register_time           43916 non-null  object
 3   recently_logged         43724 non-null  object
 4   number_of_classes_join  43916 non-null  int64 
 5   number_of_classes_out   43916 non-null  int64 
 6   learn_time              43916 non-null  object
 7   school                  10569 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.0+ MB


In [30]:
# 将是否填写学校信息设置成为0、1变量，作为后续可能有用的信息
users['是否填写学校信息']=users['school']
users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0
users.是否填写学校信息.value_counts()

C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\3679443994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\3679443994.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0


0    33347
1    10569
Name: 是否填写学校信息, dtype: int64

In [31]:
keys=login.groupby('user_id').login_time.max().index.tolist()
values=login.groupby('user_id').login_time.max().values.tolist()
login_time={}
for i in range(len(keys)):
    login_time[keys[i]]=values[i]
    
u_2=users[users.recently_logged!='--']
u_1=users[users.recently_logged=='--']
for i in range(u_1.shape[0]):
    if u_1.iloc[i,0] in login_time.keys():
        u_1.iloc[i,2]=pd.to_datetime(login_time[u_1.iloc[i,0]])
    else:
        if pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )>pd.to_datetime('2020-06-18'):
            u_1.iloc[i,2]=pd.to_datetime('2020-06-18')
            print('修改时间为最新时间')
        else:
            u_1.iloc[i,2]=pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )        
users=pd.concat([u_1,u_2])

In [24]:
users['recently_logged'].isna().sum()

192

In [33]:
# 还是以最近的时间为基准，计算出用户登陆注册行为的时间差
users = users[~users['recently_logged'].isnull()]
str_datetime(users,column_name='register_time')
str_datetime(users,column_name='recently_logged')
users['register_logged_time']=pd.to_datetime(users['recently_logged'])-pd.to_datetime(users['register_time'])
users['register_now_time']=pd.to_datetime(users['register_time']).max()-pd.to_datetime(users['register_time'])
users['logged_now_time']=pd.to_datetime(users['recently_logged']).max()-pd.to_datetime(users['register_time'])
users

C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\1779304946.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name]=pd.to_datetime(df[column_name])
C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\1779304946.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name]=df[column_name].apply(lambda x:x.strftime('%Y-%m-%d'))
C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\1779304946.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time
0,0,用户44251,2020-06-18,2020-06-18,0,0,41.25,NaN,0,0 days,0 days,0 days
1,1,用户44250,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days
2,2,用户44249,2020-06-18,2020-06-18,0,0,16.22,NaN,0,0 days,0 days,0 days
3,3,用户44248,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days
4,4,用户44247,2020-06-18,2020-06-18,0,0,1.8,NaN,0,0 days,0 days,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...
43911,43978,用户6,2018-09-11,2018-09-11,0,0,0,NaN,0,0 days,646 days,646 days
43912,43979,用户5,2018-09-10,2020-06-15,9,5,"2,116.15",NaN,0,644 days,647 days,647 days
43913,43980,用户4,2018-09-10,2020-06-05,4,0,341.2,NaN,0,634 days,647 days,647 days
43914,43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days


In [34]:
# 现在加入的班级的数量
users['number_of_classes_now']=users['number_of_classes_join']-users['number_of_classes_out']
users

C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\2085419102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['number_of_classes_now']=users['number_of_classes_join']-users['number_of_classes_out']


,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now
0,0,用户44251,2020-06-18,2020-06-18,0,0,41.25,NaN,0,0 days,0 days,0 days,0
1,1,用户44250,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days,0
2,2,用户44249,2020-06-18,2020-06-18,0,0,16.22,NaN,0,0 days,0 days,0 days,0
3,3,用户44248,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days,0
4,4,用户44247,2020-06-18,2020-06-18,0,0,1.8,NaN,0,0 days,0 days,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43911,43978,用户6,2018-09-11,2018-09-11,0,0,0,NaN,0,0 days,646 days,646 days,0
43912,43979,用户5,2018-09-10,2020-06-15,9,5,"2,116.15",NaN,0,644 days,647 days,647 days,4
43913,43980,用户4,2018-09-10,2020-06-05,4,0,341.2,NaN,0,634 days,647 days,647 days,4
43914,43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days,1


In [35]:
def nx_data(df=stu_info,group_name=['course_id','user_id']):
    # 得到共现字典
    user_dic={}  
    stu_info_data=df.groupby(group_name)['course_id'].count().unstack()
    column=stu_info_data.columns.tolist()
    for i in range(stu_info_data.shape[0]):
        user_dic[column[i]]=stu_info_data[stu_info_data[column[i]]==1].index.tolist()
        
    #构造共现矩阵
    course_name=list(set(df['course_id'].values.tolist()))
    course_data=pd.DataFrame(data=np.zeros(shape=(len(course_name),len(course_name))),index=course_name,columns=course_name)
    for value in user_dic.values():
        if len(value)==1:
            pass
        else:
            for i in range(len(value)):
                for j in range(i+1,len(value)):
                    course_data.loc[value[i],value[j]]+=1
    return (user_dic,course_data)

user_dic,course_data=nx_data()

for i,key in enumerate(user_dic.keys()):
    users.loc[i,'选课数量']=len(user_dic[key])
users

C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\2550681146.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.loc[i,'选课数量']=len(user_dic[key])
C:\Users\LiuJiacheng\AppData\Local\Temp\ipykernel_48260\2550681146.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.loc[i,'选课数量']=len(user_dic[key])


,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量
0,0.0,用户44251,2020-06-18,2020-06-18,0.0,0.0,41.25,NaN,0,0 days,0 days,0 days,0.0,2.0
1,1.0,用户44250,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0
2,2.0,用户44249,2020-06-18,2020-06-18,0.0,0.0,16.22,NaN,0,0 days,0 days,0 days,0.0,1.0
3,3.0,用户44248,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0
4,4.0,用户44247,2020-06-18,2020-06-18,0.0,0.0,1.8,NaN,0,0 days,0 days,0 days,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43913,43980.0,用户4,2018-09-10,2020-06-05,4.0,0.0,341.2,NaN,0,634 days,647 days,647 days,4.0,NaN
43914,43981.0,用户3,2018-09-04,2020-06-18,2.0,1.0,370.35,NaN,0,653 days,653 days,653 days,1.0,NaN
43915,43982.0,用户1,2018-09-03,2018-11-04,0.0,0.0,0,NaN,0,62 days,654 days,654 days,0.0,NaN
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,3.0


In [100]:
# login=pd.read_excel('用户地区切割.xlsx',index_col=0)
login_1=login.sort_values(by=['user_id','时间差1'])
login_del=login_1.user_id.drop_duplicates()
login_diff=login.iloc[list(login_del.index),:]
users_all=pd.merge(users,login_diff,how='left')
users_all.to_excel('全部信息.xlsx')
users_all=users_all.reset_index()
users_all=users_all.drop(columns=['index'])
users_all

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区
0,0.0,用户44251,2020-06-18,2020-06-18,0.0,0.0,41.25,NaN,0,0 days,0 days,0 days,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,用户44250,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国江西南昌,0.0,中国,江西,南昌
2,2.0,用户44249,2020-06-18,2020-06-18,0.0,0.0,16.22,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国北京,0.0,中国,北京,NaN
3,3.0,用户44248,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国天津,0.0,中国,天津,NaN
4,4.0,用户44247,2020-06-18,2020-06-18,0.0,0.0,1.8,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国湖北武汉,0.0,中国,湖北,武汉
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43721,43980.0,用户4,2018-09-10,2020-06-05,4.0,0.0,341.2,NaN,0,634 days,647 days,647 days,4.0,NaN,2020-06-05,中国广东广州,13.0,中国,广东,广州
43722,43981.0,用户3,2018-09-04,2020-06-18,2.0,1.0,370.35,NaN,0,653 days,653 days,653 days,1.0,NaN,2020-06-18,中国广东广州,0.0,中国,广东,广州
43723,43982.0,用户1,2018-09-03,2018-11-04,0.0,0.0,0,NaN,0,62 days,654 days,654 days,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
login_1=login.sort_values(by=['user_id','时间差1'])
login_del=login_1.user_id.drop_duplicates()
list1 = list(login_del.index)
list2 = list(login.user_id.drop_duplicates().index)
same_items = set(list1) & set(list2)
len(same_items)

In [101]:
country=login[login['国家']!='中国'].国家.value_counts().index.tolist()
country_count=login[login['国家']!='中国'].国家.value_counts().values.tolist()
login.国家.value_counts()

中国    267582
英国        84
德国        22
越南        11
荷兰         8
波兰         7
南非         3
捷克         2
泰国         2
挪威         1
瑞典         1
瑞士         1
希腊         1
Name: 国家, dtype: int64

In [138]:
from pyecharts import Line, Pie, Grid
line=Line()
line.add( "",country,country_count,mark_point=["max", "min"],mark_line=["average"])
pie = Pie("", title_pos="55%")
pie.add( "", country, country_count, radius=[45, 65],center=[74, 50],legend_pos="80%",legend_orient="vertical",rosetype="radius",is_legend_show=False,is_label_show = True)
grid = Grid(width=900)
grid.add(line, grid_right="55%")
grid.add(pie, grid_left="50%")
grid

ImportError: cannot import name 'Line' from 'pyecharts' (D:\anaconda\lib\site-packages\pyecharts\__init__.py)

In [133]:
# login=pd.read_excel('用户地区切割.xlsx',index_col=0)
provice=login[login['国家']=='中国'].省份.value_counts().index.tolist()
provice_count=login[login['国家']=='中国'].省份.value_counts().values.tolist()
bar=Bar()
bar.add('',provice,provice_count,mark_point=['max','min'],mark_line=['average'], is_datazoom_show=True)
bar

AttributeError: 'Bar' object has no attribute 'add'